In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
import pickle as pkl
import copy
import sparse

In [2]:
def to_onehot(val, cat, etc=0):

    onehot=np.zeros(len(cat))
    for ci, c in enumerate(cat):
        if val == c:
            onehot[ci]=1

    if etc==1 and np.sum(onehot)==0:
        print(val)

    return onehot

In [3]:
def atomFeatures(a, ri_a, symbol_list, hybrid_list):

    def _ringSize_a(a, rings):
        onehot = np.zeros(6)
        aid = a.GetIdx()
        for ring in rings:
            if aid in ring and len(ring) <= 8:
                onehot[len(ring) - 3] += 1

        return onehot

    v1 = to_onehot(a.GetSymbol(), symbol_list, 1)
    v2 = to_onehot(str(a.GetHybridization()), hybrid_list, 1)

    v3 = [a.GetAtomicNum(), a.GetDegree(), a.GetFormalCharge(), a.GetTotalNumHs(), atom.GetImplicitValence(), a.GetNumRadicalElectrons(), int(a.GetIsAromatic())]
    v4 = _ringSize_a(a, ri_a)

    v5 = np.zeros(3)
    try:
        tmp = to_onehot(a.GetProp('_CIPCode'), ['R','S'], 1)
        v5[0] = tmp[0]
        v5[1] = tmp[1]
    except:
        v5[2]=1

    v5 = v5[:2]

    return np.concatenate([v1,v2,v3,v4,v5], axis=0)

In [4]:
def bondFeatures(bbs, samering, shortpath):

    if len(bbs)==1:
        v1 =  to_onehot(str(bbs[0].GetBondType()), ['SINGLE', 'DOUBLE', 'TRIPLE', 'AROMATIC'], 1)
        v2 = to_onehot(str(bbs[0].GetStereo()), ['STEREOZ', 'STEREOE','STEREOANY','STEREONONE'], 1)
        v2 = v2[:2]
        v3 = [int(bbs[0].GetIsConjugated()), int(bbs[0].IsInRing()), samering, shortpath]
    else:
        v1 = np.zeros(4)
        v2 = np.zeros(2)
        v3 = [0, 0, samering, shortpath]

    return np.concatenate([v1,v2,v3], axis=0)

In [5]:
loaddir = '/home/bb596/rds/hpc-work/dl4chem/'
savedir = '/home/bb596/rds/hpc-work/dl4chem/'
data='QM9'

if data == 'COD' :
    symbol_list = ['C','N','O','F','Cl','Br','I','S','B','Si','P','Te','Se','Ge','As']
    hybrid_list = ['SP','SP2','SP3','SP3D','SP3D2']
    n_max=50
    atom_dim=35
elif data == 'QM9' :
    symbol_list = ['C','N','O','F']
    hybrid_list = ['SP','SP2','SP3']
    n_max=9
    atom_dim=22

n_min=2
edge_dim=10

In [6]:
[mollist, smilist] = pkl.load(open(loaddir+data+'_molset_all.p','rb'))

In [13]:
for i in range(len(smilist)) :
    smi = smilist[i]
    mol = mollist[i]
    molsmi = Chem.MolToSmiles(mol)
    if smi != molsmi :
        print(smi)
        print(molsmi + '\n')

C1COCC[C@]2(C1)CC2
C1COCCC2(C1)CC2

C[C@@H](C)Nc1[nH]nn[nH+]1
CC(C)Nc1[nH]nn[nH+]1

C[C@@]1(C)CN=COC1
CC1(C)CN=COC1

F[C@@](F)(F)[C@H]1CN[CH+][NH2+]1
FC(F)(F)[C@H]1CN[CH+][NH2+]1

CC#C[C@@]1(C=O)CCC1
CC#CC1(C=O)CCC1

C[C@]1(C)[C@H]2C[C@H]2[C@]1(C)C
CC1(C)[C@H]2C[C@H]2C1(C)C

C#C[C@]1(COC)CC1
C#CC1(COC)CC1

C[C@@]1(C)CC(=O)C([NH-])O1
CC1(C)CC(=O)C([NH-])O1

C/C(=N\O)[C@@H](CO)CO
C/C(=N\O)C(CO)CO

C[C@]1(C)OC[C@H]2C[C@H]21
CC1(C)OC[C@H]2C[C@H]21

O=C1N[C@@H]2C[C@@H](C2)N1
O=C1N[C@H]2C[C@H](C2)N1

C[C@@H](C)[N@H+]1[C@@H](C)[C@H]1C#N
CC(C)[N@H+]1[C@@H](C)[C@H]1C#N

C[C@@](O)(C=O)[C@H]1COC1
C[C@@](O)(C=O)C1COC1

CCN[C@@](C)(C)C#N
CCNC(C)(C)C#N

C[C@@H](CO)C[C@@](C)(C)O
C[C@@H](CO)CC(C)(C)O

CNC(=O)C#C[C@H]1CC1
CNC(=O)C#CC1CC1

O=C1OC[C@H]([C@H]2CC2)O1
O=C1OC[C@H](C2CC2)O1

CC/C(=N\O)[C@@](C)(C)O
CC/C(=N\O)C(C)(C)O

O[C@]12[C@]3(CC3)[C@@H]3[C@]14O[C@@]324
O[C@]12[C@@]3(CC3)[C@@H]3[C@]14O[C@@]324

CCC(=O)[C@@H](O)[C@@H](C)C
CCC(=O)[C@@H](O)C(C)C

CCC[C@]1(C#N)CCC1
CCCC1(C#N)CCC1

C[C@H]1O[C@]

In [9]:
mol = mollist[0]

In [11]:
Chem.MolToSmiles(mol)

'C#C[C@@H](O)CC(=O)C=O'

In [12]:
smilist[0]

'C#C[C@@H](O)CC(=O)C=O'

In [7]:
mollist2 = []
smilist2 = []

for i in range(len(smilist)) :
    if i % 1000 == 0: print(i, flush=True)
        
    smi = smilist[i]
    mol = mollist[i]
        
    Chem.rdmolops.AssignAtomChiralTagsFromStructure(mol)
    Chem.rdmolops.AssignStereochemistry(mol)
    
    if mol.GetNumHeavyAtoms() < n_min or mol.GetNumHeavyAtoms() > n_max:
        print('error')
        break
        
    mollist2.append(mol)
    smilist2.append(smi)
    
mollist2 = np.array(mollist2)
smilist2 = np.array(smilist2)

molset_fname = savedir + data + '_molset_' + str(n_max) + '.p'

with open(molset_fname,'wb') as f:
    pkl.dump([mollist2, smilist2], f)
    
del mollist2, smilist2

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000


In [11]:
nodes = []
for i in range(len(smilist)) :
    if i % 1000 == 0: print(i, flush=True)

    smi = smilist[i]
    mol = mollist[i]
    
    Chem.rdmolops.AssignAtomChiralTagsFromStructure(mol)
    Chem.rdmolops.AssignStereochemistry(mol)
    
    if mol.GetNumHeavyAtoms() < n_min or mol.GetNumHeavyAtoms() > n_max:
        print('error')
        break
        
    n = mol.GetNumAtoms()
    ri = mol.GetRingInfo()
    ri_a = ri.AtomRings()
    
    pos = mol.GetConformer().GetPositions()

    assert n==pos.shape[0]

    node = np.zeros((n_max, atom_dim))
    
    for j in range(n):
        atom = mol.GetAtomWithIdx(j)
        node[j, :]=atomFeatures(atom, ri_a, symbol_list, hybrid_list)
        
    nodes.append(np.array(node, dtype=int))
    
nodes = np.array(nodes, dtype=int)
nodes = sparse.COO.from_numpy(nodes)

nodes_fname = savedir + data+'_nodes_'+str(n_max)+'.p'

with open(nodes_fname,'wb') as f:
    pkl.dump(nodes, f)
    
del nodes

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000


In [12]:
masks = []
for i in range(len(smilist)) :
    if i % 1000 == 0: print(i, flush=True)

    smi = smilist[i]
    mol = mollist[i]
    
    Chem.rdmolops.AssignAtomChiralTagsFromStructure(mol)
    Chem.rdmolops.AssignStereochemistry(mol)
    
    if mol.GetNumHeavyAtoms() < n_min or mol.GetNumHeavyAtoms() > n_max:
        print('error')
        break
        
    n = mol.GetNumAtoms()
    ri = mol.GetRingInfo()
    ri_a = ri.AtomRings()
    
    pos = mol.GetConformer().GetPositions()

    assert n==pos.shape[0]

    mask = np.zeros((n_max, 1))
    
    for j in range(n):
        atom = mol.GetAtomWithIdx(j)
        mask[j, 0]=1
        
    masks.append(np.array(mask, dtype=int))
    
masks = np.array(masks, dtype=int)
masks = sparse.COO.from_numpy(masks)

masks_fname = savedir + data+'_masks_'+str(n_max)+'.p'

with open(masks_fname,'wb') as f:
    pkl.dump(masks, f)
    
del masks

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000


In [13]:
edges = []
for i in range(len(smilist)) :
    if i % 1000 == 0: print(i, flush=True)

    smi = smilist[i]
    mol = mollist[i]
    
    Chem.rdmolops.AssignAtomChiralTagsFromStructure(mol)
    Chem.rdmolops.AssignStereochemistry(mol)
    
    if mol.GetNumHeavyAtoms() < n_min or mol.GetNumHeavyAtoms() > n_max:
        print('error')
        break
        
    n = mol.GetNumAtoms()
    ri = mol.GetRingInfo()
    ri_a = ri.AtomRings()
    
    pos = mol.GetConformer().GetPositions()

    assert n==pos.shape[0]
    
    edge = np.zeros((n_max, n_max, edge_dim))
    
    for j in range(n-1):
        for k in range(j+1, n):
            molpath = Chem.GetShortestPath(mol, j, k)
            shortpath = len(molpath) - 1
            assert shortpath>0

            samering = 0
            for alist in ri_a:
                if j in alist and k in alist:
                    samering = 1

            bond = [mol.GetBondBetweenAtoms(molpath[mm], molpath[mm+1]) for mm in range(shortpath)]

            edge[j, k, :] = bondFeatures(bond, samering, shortpath)
            edge[k, j, :] = edge[j, k, :]

        
    edges.append(np.array(edge, dtype=int))
    
edges = np.array(edges, dtype=int)
edges = sparse.COO.from_numpy(edges)

edges_fname = savedir + data+'_edges_'+str(n_max)+'.p'

with open(edges_fname,'wb') as f:
    pkl.dump(edges, f)
    
del edges

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000


In [14]:
dist_mats = []
for i in range(len(smilist)) :
    if i % 1000 == 0: print(i, flush=True)

    smi = smilist[i]
    mol = mollist[i]
    
    Chem.rdmolops.AssignAtomChiralTagsFromStructure(mol)
    Chem.rdmolops.AssignStereochemistry(mol)
    
    if mol.GetNumHeavyAtoms() < n_min or mol.GetNumHeavyAtoms() > n_max:
        print('error')
        break
        
    n = mol.GetNumAtoms()
    ri = mol.GetRingInfo()
    ri_a = ri.AtomRings()
    
    pos = mol.GetConformer().GetPositions()

    assert n==pos.shape[0]
    
    dist_mat = np.zeros((n_max, n_max))
    dist_mat[:n, :n] = euclidean_distances(pos)
        
    dist_mats.append(np.array(dist_mat))
    
dist_mats = np.array(dist_mats)
#dist_mats = sparse.COO.from_numpy(dist_mats)

dist_mats_fname = savedir + data+'_dist_mats_'+str(n_max)+'.p'

with open(dist_mats_fname,'wb') as f:
    pkl.dump(dist_mats, f)
    
del dist_mats

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000


In [15]:
positions = []
for i in range(len(smilist)) :
    if i % 1000 == 0: print(i, flush=True)

    smi = smilist[i]
    mol = mollist[i]
    
    Chem.rdmolops.AssignAtomChiralTagsFromStructure(mol)
    Chem.rdmolops.AssignStereochemistry(mol)
    
    if mol.GetNumHeavyAtoms() < n_min or mol.GetNumHeavyAtoms() > n_max:
        print('error')
        break
        
    n = mol.GetNumAtoms()
    ri = mol.GetRingInfo()
    ri_a = ri.AtomRings()
    
    pos = mol.GetConformer().GetPositions()

    assert n==pos.shape[0]
    
    pos2 = np.zeros((n_max, 3))
    pos2[:n] = pos
        
    positions.append(np.array(pos2))
    
positions = np.array(positions)
#positions = sparse.COO.from_numpy(positions)

positions_fname = savedir + data+'_positions_'+str(n_max)+'.p'

with open(positions_fname,'wb') as f:
    pkl.dump(positions, f)
    
del positions

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
